# Scientific Computing Project 2
# Brian Livian, Yonah Moise

In [525]:
import numpy as np
import math
# Given satellite positions
satmatrix1 = np.array([
    [15600, 7540, 20140],
    [18760, 2750, 18610],
    [17610, 14630, 13480],
    [19170, 610, 18390]])

# Given time
time = [.07074, .07220, .07690, .07242]

# Given initial vector
initial = np.array([0,0,6370, 0])

c    = 299792.458



In [526]:
# Newton algorithm
def Newton_system(r, Dr, x0, N = 100, eps = .00001):
    x=x0
    r_value=r(x0)
    Gradr_norm = np.linalg.norm(np.dot(r(x0),Dr(x0)),ord=2)
    steps = 0
    while abs(Gradr_norm) > eps and steps < N: 
        v=np.linalg.lstsq(Dr(x), r_value,rcond=None)[0]
        #print('v=',v)
        x = x-v
        #print('x=',x)
        r_value = r(x)
        Dr_value = Dr(x) 
        Gradr_norm = np.linalg.norm(np.dot(r_value,Dr_value),ord=2)
        #print('Gradr_norm =',Gradr_norm)
        steps = steps + 1
    # Either a solution is found, or too many iterations
    if abs(Gradr_norm) > eps:
        steps = -1
    return x, steps


In [527]:
def Newton_system1(satmatrix, g, t):
    # Define F and DF for Newtown algorithm
    def F(x):
        return np.array(
            [(g[0] - satmatrix[0][0])**2  + (g[1] - satmatrix[0][1]) **2 + (g[2] - satmatrix[0][2]) **2 - ((c* (t[0] - g[3]))**2),
            (g[0] - satmatrix[1][0])**2  + (g[1] - satmatrix[1][1]) **2 + (g[2] - satmatrix[1][2]) **2 - ((c* (t[1] - g[3]))**2),
            (g[0] - satmatrix[2][0])**2  + (g[1] - satmatrix[2][1]) **2 + (g[2] - satmatrix[2][2]) **2 - ((c* (t[2] - g[3]))**2),
            (g[0] - satmatrix[3][0])**2  + (g[1] - satmatrix[3][1]) **2 + (g[2] - satmatrix[3][2]) **2 - ((c* (t[3] - g[3]))**2)]
        )
        
    def DF(x):
        return np.array(
            [[2*(g[0] - satmatrix[0][0]), 2*(g[1]-satmatrix[0][1]), 2*(g[2] - satmatrix[0][2]), (2* (c**2) * (t[0] - g[3]) )],
             [2*(g[0] - satmatrix[1][0]), 2*(g[1]-satmatrix[1][1]), 2*(g[2] - satmatrix[1][2]), (2* (c**2) * (t[1] - g[3]) )],
             [2*(g[0] - satmatrix[2][0]), 2*(g[1]-satmatrix[2][1]), 2*(g[2] - satmatrix[2][2]), (2 * (c**2) * (t[2] - g[3]) )],
             [2*(g[0] - satmatrix[3][0]), 2*(g[1]-satmatrix[3][1]), 2*(g[2] - satmatrix[3][2]), (2* (c**2) * (t[3] - g[3]) )]
        
            
            ])
    
    tol = (1 * (10)**-8)
    
    h, n = Newton_system(F, DF, x0 = g , N = 1, eps=0.000001)
    return (h)
   
    
    


In [512]:
# Location of reciever
Newton_system1(satmatrix1, initial, time)


array([-4.17733669e+01, -1.67510571e+01,  6.37518313e+03, -3.26271451e-03])

# Part 2

In [513]:
from math import sin, cos, sqrt, pi
p = 26570

# Phi and theta chose arbitrarily in northern hemisphere
phi2 = [pi/8, pi/4, 3*pi/8, pi/2]
theta2 = [pi/2, pi, 3*pi/2, 2*pi]
# Given location
loc = [0,0,6370, .0001]

R = []
t2 = []
a = []
b = []
c = []

# Find cartesian coordinates of satellites   
for i, j in zip(phi2, theta2):
    a.append(p*cos(i)*cos(j))
    b.append(p*cos(i)*sin(j))
    c.append(p*sin(i))

arr = np.array([a, b, c])
satmatrix2 = arr.T
print(satmatrix2)


# Calculate distances
def functionforR(satmatrix2):
    
    for sat in satmatrix2:
        R.append(sqrt(sat[0]**2 + sat[1]**2 + (sat[2] - 6370)**2))
    return R

functionforR(satmatrix2)

# Calculate times
def functionfort(R):
    
    for r in R:
        t2.append(.0001 + (r/299792.458) )
    return t2

functionfort(R)

t2 = np.array(t2)
print(R)
print(t2)


[[ 1.50309959e-12  2.45474792e+04  1.01678988e+04]
 [-1.87878272e+04  2.30084524e-12  1.87878272e+04]
 [-1.86781271e-12 -1.01678988e+04  2.45474792e+04]
 [ 1.62694327e-12 -3.98486174e-28  2.65700000e+04]]
[24839.54044088253, 22520.76556816281, 20828.03195843935, 20200.0]
[0.08295579 0.07522119 0.06957484 0.06747995]


In [518]:
# If you cannot run this cell, run the first 3 cells in part 1 and run this one again
xyz2 = Newton_system1(satmatrix2, loc, t2)
print(xyz2)

[1.34434840e-12 2.81375117e-12 6.37000000e+03 1.00000000e-04]


In [519]:
q = 1.0e-08
# All permuations of possible errors in time
# There are 4 satellites with 2 possible errors each
# Thus there are 2^4 = 16 permutations
dtarray = np.array([
    [q, q, q, q], 
    [-q, q, q, q], 
    [q, -q, q, q,], 
    [q, q, -q, q], 
    [q, q, q, -q], 
    [-q, -q, q, q,], 
    [q, q, -q, -q], 
    [-q, q, q, -q], 
    [q, -q, -q, q], 
    [-q ,q, -q, q], 
    [q, -q, q, -q], 
    [q, -q, -q, -q],
    [-q, q, -q, -q], 
    [-q, -q, q, -q], 
    [-q, -q, -q, q], 
    [-q, -q, -q, -q]
    
])

In [534]:
# Calculate new times with every permutation of dt
tnewarray2 = []
for dt in dtarray:
    tnewarray2.append(t2 + dt)
tnewarray2 = np.array(tnewarray2)
print('tnew: ' + str(tnewarray2))
print('')

# Calculate new position with each permuation of dt
xyzarray2 = []
for i in tnewarray2:
    xyzarray2.append(Newton_system1(satmatrix2, loc, i))
print('xyz array: ' + str(xyzarray2))
print('')

# Calculate change in posistion for each new xyz
dxyzarray2 = []
for i in xyzarray2:
    dxyzarray2.append(xyz2 - i)
print('dxyz: ' + str(dxyzarray2))  
print('')

# Calculate each length of dxyz 
normdxyz2 = []
for i in dxyzarray2:
    normdxyz2.append(sqrt(i[0]**2 + i[1]**2 +i[2]**2))
print('Norm dxyz: ' + str(normdxyz2))
print('')

# norm of dt multiplied by speed of light
normdt = sqrt((q**2)*4)
normdt = normdt * 299792.458

# Calculate error magnification for each dxyz
errormag2 = []
for i in normdxyz2:
    errormag2.append(i/normdt)
print('Error magnification: ' + str(errormag2))
print('')

print('Max forward error: ' + str(np.max(normdxyz2)))

# Max error magnification
print('ConditionN: ' + str(np.max(errormag2)))

tnew: [[0.0829558  0.0752212  0.06957485 0.06747996]
 [0.08295578 0.0752212  0.06957485 0.06747996]
 [0.0829558  0.07522118 0.06957485 0.06747996]
 [0.0829558  0.0752212  0.06957483 0.06747996]
 [0.0829558  0.0752212  0.06957485 0.06747994]
 [0.08295578 0.07522118 0.06957485 0.06747996]
 [0.0829558  0.0752212  0.06957483 0.06747994]
 [0.08295578 0.0752212  0.06957485 0.06747994]
 [0.0829558  0.07522118 0.06957483 0.06747996]
 [0.08295578 0.0752212  0.06957483 0.06747996]
 [0.0829558  0.07522118 0.06957485 0.06747994]
 [0.0829558  0.07522118 0.06957483 0.06747994]
 [0.08295578 0.0752212  0.06957483 0.06747994]
 [0.08295578 0.07522118 0.06957485 0.06747994]
 [0.08295578 0.07522118 0.06957483 0.06747996]
 [0.08295578 0.07522118 0.06957483 0.06747994]]

xyz array: [array([3.77559015e-12, 1.49129393e-12, 6.37000000e+03, 1.00010000e-04]), array([2.91857714e-03, 1.41483329e-03, 6.36999457e+03, 9.99918958e-05]), array([-7.18715868e-03,  1.49129512e-12,  6.37000000e+03,  1.00010000e-04]), array

# Part 3

In [524]:
# Phi and theta chose arbitrarily in northern hemisphere
# Phi and theta chosen between 5% of each other
phi3 = [pi/4, .01 + pi/4, .02 + pi/4, .03 + pi/4]
theta3 = [pi, .05 + pi, .10 + pi, .15 + pi]
loc = [0,0,6370, .0001]

R = []
t3 = []
a = []
b = []
c = []
    
for x, y in zip(phi3, theta3):
    a.append(p*cos(x)*cos(y))
    b.append(p*cos(x)*sin(y))
    c.append(p*sin(x))

arr3 = np.array([a, b, c])
satmatrix3 = arr3.T
print(satmatrix3)

def functionforR(satmatrix3):
    
    for sat in satmatrix3:
        R.append(sqrt(sat[0]**2 + sat[1]**2 + (sat[2] - 6370)**2))
        
    return R

functionforR(satmatrix3)
    
def functionfort(R):
    
    for r in R:
        t3.append(.0001 + (r/299792.458) )
    return t3

functionfort(R)
t3 = np.array(t3)

print(R)
print(t3)


[[-1.87878272e+04  2.30084524e-12  1.87878272e+04]
 [-1.85757687e+04 -9.29563202e+02  1.89747629e+04]
 [-1.83163732e+04 -1.83776730e+03  1.91598012e+04]
 [-1.80112790e+04 -2.72213857e+03  1.93429236e+04]]
[22520.76556816281, 22467.82856069337, 22415.30576047979, 22363.205356617425]
[0.07522119 0.07504461 0.07486941 0.07469562]


In [528]:
# If you cannot run this cell, run the first 3 cells in part 1 and run this one again
xyz3 = Newton_system1(satmatrix3, loc, t3)
print('xyz: ' + str(xyz3))

xyz: [ 3.17704725e-09 -1.03910811e-08  6.37000000e+03  9.99999999e-05]


In [535]:
# Calculate new times with every permutation of dt
tnewarray3 = []
for dt in dtarray:
    tnewarray3.append(t3 + dt)
tnewarray3 = np.array(tnewarray3)
print('tnew: ' + str(tnewarray3))
print('')

# Calculate new position with each permuation of dt
xyzarray3 = []
for i in tnewarray3:
    xyzarray3.append(Newton_system1(satmatrix3, loc, i))
print('xyz: ' + str(xyzarray))
print('')

# Calculate change in posistion for each new xyz
dxyzarray3 = []
for i in xyzarray3:
    dxyzarray3.append(xyz3 - i)
print('dxyz: ' + str(dxyzarray3))
print('')

# Calculate each length of dxyz 
normdxyz3 = []
for i in dxyzarray3:
    normdxyz3.append(sqrt(i[0]**2 + i[1]**2 +i[2]**2))
print('Norm of dxzy: ' + str(normdxyz3))
print('')

# norm of dt multiplied by speed of light
normdt = sqrt((q**2)*4)
normdt = normdt * 299792.458

# Calculate error magnification for each dxyz
errormag3 = []
for i in normdxyz3:
    errormag3.append(i/normdt)
print('Error magnifications: ' + str(errormag3))
print('')

# Max forward error
print('Max forward error: ' + str(np.max(normdxyz3)))

# Max error magnification
print('ConditionN: ' + str(np.max(errormag3)))

tnew: [[0.0752212  0.07504462 0.07486942 0.07469563]
 [0.07522118 0.07504462 0.07486942 0.07469563]
 [0.0752212  0.0750446  0.07486942 0.07469563]
 [0.0752212  0.07504462 0.0748694  0.07469563]
 [0.0752212  0.07504462 0.07486942 0.07469561]
 [0.07522118 0.0750446  0.07486942 0.07469563]
 [0.0752212  0.07504462 0.0748694  0.07469561]
 [0.07522118 0.07504462 0.07486942 0.07469561]
 [0.0752212  0.0750446  0.0748694  0.07469563]
 [0.07522118 0.07504462 0.0748694  0.07469563]
 [0.0752212  0.0750446  0.07486942 0.07469561]
 [0.0752212  0.0750446  0.0748694  0.07469561]
 [0.07522118 0.07504462 0.0748694  0.07469561]
 [0.07522118 0.0750446  0.07486942 0.07469561]
 [0.07522118 0.0750446  0.0748694  0.07469563]
 [0.07522118 0.0750446  0.0748694  0.07469561]]

xyz: [array([ 6.76758048e-09, -2.33749517e-08,  6.37000000e+03,  1.00010000e-04]), array([ 1.44567010e+01, -4.73756018e+01,  6.15437779e+03, -3.36823371e-04]), array([-4.00295987e+01,  1.40089403e+02,  7.00424049e+03,  1.37793130e-03]), arr